# 5-OpenJij GPU

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/OpenJij/OpenJijTutorial/blob/master/source/en/005-OpenJijGPU.ipynb)

In this section, we introduce a method of Simulated Quantum Annealing (SQA) with GPU.

## Setup and installation in Google Colab

The average PC may not have a CUDA environment, or may not have an NVIDIA GPU. However, with **Google colaboratory**, anyone can use GPGPUs.

### GPU Settings

We open Jupyter Notebook file we want to run on GPU with Google Colab.

```
Runtime > Change runtime type > choose Hardware accelerator GPU
```

This setting allows you to run Jupyter Notebook on GPU.

### Installation of OpenJij 

To use GPU modules from OpenJij, one has to build from source codes when installing OpenJij.
We first update CMake, which is needed for the installation, to the latest version.

In [ ]:
!pip install -U cmake

After that, we uninstall & reinstall OpenJij with option `--no-binary :all:`, which forces to build from source codes during installation.

In [ ]:
!pip uninstall openjij -y
!pip install openjij --no-binary :all: --no-cache-dir

## Calculation with GPUSQA

OpenJij GPUs support Monte Carlo calculations on Chimeragraphs.  
We make GPUSQASampler instance at first.
If the notebook gives an error "No modules found. ...", please try to restart the session so that the reinstalled version of OpenJij is loaded correctly.

In [3]:
import openjij as oj

sampler = oj.GPUSQASampler(num_reads=100, unit_num_L=2)

`unit_num_L` specifies the length of one side of the two-dimensional lattice of chimera-unit cells.  
In the example above, we have created a system with 2x2=4 chimera-unit cells.

Since a chimera-unit cell consists of one 8-bit, the total number of bits is calculated as follows.
  
${\rm total\_num} = {\rm unit\_num\_L}^2 \times 8$

The GPUSQASampler perform SQA calculations for the size of the system of chimeragraphs created here, even if you specify fewer variables, such as sampler_qubo, later. We note that `unit_num_L` is not too large.
Also, `unit_num_L` is not specified in the constructor but is used as an argument to run `sample_qubo`.

## Execution

Next we execute SQA. As with Sampler before we introduce, we can use `sample_qubo` (or `sample_ising`).
However, if you specify an interaction for a subscript that does not satisfy the chimeragraph, error occurs.

In [4]:
Q = {(0, 4): -1, (0, 5): -1, (4, 12): 1}
response = sampler.sample_qubo(Q)
response

Response(rec.array([([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0], -2., 1),
           ([1, 1, 1, 0], -2., 1), ([1, 1, 1, 0

## Embedding the All-Coupled Model in Chimeragraph 

We introduced the calculation of only those interactions that exist on the chimeragraph. Below is script for embedding comples interactions into chimeragraph, such as problems of all-coupled.

In order to embed in chimeragraph, we use `EmbeddingComposite` of `dwave`. We install `dwave-system` with `pip`.

In [ ]:
!pip install dwave-system

Then we rewrite the sampler part of the script as follows.

In [15]:
from dwave.system.composites import EmbeddingComposite

gpusqa = oj.GPUSQASampler(num_reads=100, unit_num_L=2)
sampler = EmbeddingComposite(gpusqa)